In [3]:
import json

import requests

base_url = "https://rmysqpqqjg.execute-api.us-east-1.amazonaws.com/dev/"
endpoint = "create_job"
api_url = f"{base_url}{endpoint}"

request_body = {
    "job_name": "example_job",
    "works": [
        {
            "work_id": "work1",
            "s3_uris": ["s3://bucket/file1.txt", "s3://bucket/file2.txt"]
        },
        {
            "work_id": "work2",
            "s3_uris": ["s3://bucket/file3.txt"]
        }
    ]
}

# Headers
headers = {"Content-Type": "application/json"}

try:
    # Make the POST request
    response = requests.post(api_url, data=json.dumps(request_body), headers=headers)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        print("API Response:", data)
    else:
        print(f"Error: API request failed with status code {response.status_code}")
        print("Response:", response.text)

except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")

API Response: {'message': 'Success'}
